In [1]:
import os
from PIL import Image, ImageDraw, ImageFont
import exifread
from datetime import datetime

# Function to extract GPS data from EXIF
def extract_gps_info(tags):
    gps_info = {}
    for tag in tags.keys():
        if tag.startswith('GPS'):
            gps_info[tag] = tags[tag]

    gps_latitude = gps_info.get('GPS GPSLatitude')
    gps_latitude_ref = gps_info.get('GPS GPSLatitudeRef')
    gps_longitude = gps_info.get('GPS GPSLongitude')
    gps_longitude_ref = gps_info.get('GPS GPSLongitudeRef')

    if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
        lat = [float(x.num) / float(x.den) for x in gps_latitude.values]
        lon = [float(x.num) / float(x.den) for x in gps_longitude.values]

        latitude = lat[0] + lat[1] / 60.0 + lat[2] / 3600.0
        if gps_latitude_ref.values[0] != 'N':
            latitude = -latitude

        longitude = lon[0] + lon[1] / 60.0 + lon[2] / 3600.0
        if gps_longitude_ref.values[0] != 'E':
            longitude = -longitude

        return latitude, longitude
    return None, None

# Function to add timestamp and GPS info to image
def add_timestamp_and_gps(image_path, output_dir):
    with open(image_path, 'rb') as image_file:
        img = Image.open(image_file)
        img = img.convert('RGB')

        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype("arial.ttf", 50)

        with open(image_path, 'rb') as f:
            tags = exifread.process_file(f)
            datetime_original = tags.get('EXIF DateTimeOriginal')
            if datetime_original:
                timestamp = str(datetime_original)
                print(f"Timestamp for {image_path}: {timestamp}")
            else:
                timestamp = "Timestamp: Not available"
                print(f"No timestamp found for {image_path}")

        latitude, longitude = extract_gps_info(tags)
        if latitude is not None and longitude is not None:
            gps_text = f"GPS: {latitude:.5f}, {longitude:.5f}"
            print(f"GPS for {image_path}: {gps_text}")
        else:
            gps_text = "GPS: Not available"
            print(f"No GPS data found for {image_path}")

        # Positioning the text
        timestamp_position = (10, img.size[1] - 120)  
        gps_position = (10, img.size[1] - 180) 

        # Adding timestamp
        draw.text(timestamp_position, f"Timestamp: {timestamp}", font=font, fill="yellow")
        # Adding GPS info
        draw.text(gps_position, gps_text, font=font, fill="yellow")

        # Save the modified image
        modified_image_path = os.path.join(output_dir, os.path.basename(image_path))
        img.save(modified_image_path)

# Function to process a whole folder
def process_folder(folder_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            add_timestamp_and_gps(image_path, output_folder)
            print(f"Processed {filename}")

# Example usage
input_folder = 'C:/Users/info/Desktop/Trial_code/Photos'  # Replace with your input folder path
output_folder = 'C:/Users/info/Desktop/Trial_code/Processed_Photos'  # Replace with your output folder path
process_folder(input_folder, output_folder)

Timestamp for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131445263.jpg: 2024:08:19 14:14:45
GPS for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131445263.jpg: GPS: 51.49758, -0.13291
Processed PXL_20240819_131445263.jpg
Timestamp for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131448571.jpg: 2024:08:19 14:14:48
GPS for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131448571.jpg: GPS: 51.49758, -0.13291
Processed PXL_20240819_131448571.jpg
Timestamp for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131451748.jpg: 2024:08:19 14:14:51
GPS for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131451748.jpg: GPS: 51.49758, -0.13291
Processed PXL_20240819_131451748.jpg
Timestamp for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131458181.jpg: 2024:08:19 14:14:58
GPS for C:/Users/info/Desktop/Trial_code/Photos\PXL_20240819_131458181.jpg: GPS: 51.49767, -0.13255
Processed PXL_20240819_131458181.jpg
Timestamp for C:/Users/info/Desktop/Trial_co